In [83]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from varsvm import nonneg_driftsvm

ModuleNotFoundError: No module named 'varsvm'

In [4]:
f_data=pd.read_csv('dataset.csv')
f_data.head()

,Batting Innings,Batting Position,Runs Scored,Balls Faced,Batting Strikerate,Out/NotOut,4s,6s,50s,100s,...,5 wicket hauls,Wickets in Powerplay,Wickets in Middle Overs,Wickets in Death Overs,Economy in Powerplay,Economy in Middle Overs,Economy in Death Overs,Catches or Runouts as Fielder,Match Result,Man of the match
0,1.0,0.0,10.0,12.0,83.33,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,-1.0,-1.0,5.52,0.0,1.0,0.0
1,2.0,8.0,14.0,30.0,46.67,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.00,1.0,0.0,0.0
2,1.0,0.0,12.0,19.0,63.16,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.00,0.0,0.0,0.0
3,1.0,0.0,4.0,2.0,200.00,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,-1.0,7.00,0.0,0.0,0.0
4,2.0,2.0,51.0,39.0,130.77,1.0,4.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,-1.0,14.00,0.0,0.0,0.0


In [15]:
f_data['Balls Bowled'].mean()

11.017553990068137

In [5]:
f_data.columns


Index(['Batting Innings', 'Batting Position', 'Runs Scored', 'Balls Faced',
       'Batting Strikerate', 'Out/NotOut', '4s', '6s', '50s', '100s',
       'Runs in Powerplay', 'Runs in Middle Overs', 'Runs in Death Overs',
       'Strikerate in first 20 balls', 'Strikerate in 20 to 30 balls',
       'Strikerate after 30 balls', 'Bowling Innings', 'Runs Given',
       'Balls Bowled', 'Wickets Taken', 'Economy Rate', 'Bowling Strikerate',
       '3 wicket hauls', '5 wicket hauls', 'Wickets in Powerplay',
       'Wickets in Middle Overs', 'Wickets in Death Overs',
       'Economy in Powerplay', 'Economy in Middle Overs',
       'Economy in Death Overs', 'Catches or Runouts as Fielder',
       'Match Result', 'Man of the match'],
      dtype='object')

In [49]:
t_data=f_data.loc[(f_data['Balls Bowled']<6) & (f_data['Balls Faced']>10)]

In [74]:
data=t_data[['Runs Scored', 'Balls Faced',
       'Batting Strikerate', '4s', '6s','Runs in Powerplay', 'Runs in Middle Overs', 'Runs in Death Overs',
       'Strikerate in first 20 balls', 'Strikerate in 20 to 30 balls',
       'Strikerate after 30 balls','Match Result', 'Man of the match']]

In [81]:
data.head()

,Runs Scored,Balls Faced,Batting Strikerate,4s,6s,Runs in Powerplay,Runs in Middle Overs,Runs in Death Overs,Strikerate in first 20 balls,Strikerate in 20 to 30 balls,Strikerate after 30 balls,Match Result,Man of the match
1,-0.423077,-0.193548,-0.419645,-0.447368,-0.500000,-0.454023,-0.406542,-0.500000,-0.454034,-0.305556,-0.50000,-0.5,0.0
2,-0.435897,-0.370968,-0.373135,-0.447368,-0.500000,-0.396552,-0.471963,-0.500000,-0.327586,-0.500000,-0.50000,-0.5,0.0
9,-0.416667,-0.354839,-0.339741,-0.394737,-0.500000,-0.488506,-0.369159,-0.500000,-0.275862,-0.500000,-0.50000,-0.5,0.0
12,0.038462,0.177419,-0.093626,-0.184211,-0.038462,-0.270115,-0.266355,0.183333,-0.384828,-0.415833,-0.11595,0.5,0.0
20,-0.230769,0.048387,-0.275490,-0.447368,-0.346154,-0.316092,-0.238318,-0.500000,-0.350586,-0.433333,-0.34445,-0.5,0.0


In [76]:
for col in data.columns:
    for i in data.index:
        if data[col][i]<0:
            data[col][i]=0;

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


- First we attempt to create a linear model where the X is a feature vector and the y is the man of the match. 
- We consider all features which when increases increases the chance of being a better player. 
- So now we learn a linear classification model where the weights are constrained to be non negative so that the weights kind of give the fraction of contribution towards deciding the player as a star performer. This is because we cant say a player was a star based on one statistic so we try to gauge which feature kind of contributes more
- We first attempt to use a SVM rather than sigmoid based model. So we will centre the data btwn [-0.5,0.5]



In [78]:
for col in data.columns:
    
    maxer=data[col].max()
    miner=data[col].min()
    data[col]=(data[col]-miner)/(maxer-miner)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [80]:
for col in data.columns:
    if col=='Man of the match':
        break
    data[col]-=0.5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
